In [79]:
##Determing optimal numbers and 
print("hello")

hello


In [80]:
pip install scikeras

Note: you may need to restart the kernel to use updated packages.


In [81]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import datetime
import tensorflow
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow.keras.models import Model





In [82]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [83]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [84]:
#Encode categorial Variables means convert text into numbers
label_encoder_gender=LabelEncoder()
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])#0 -female male for  1
data


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [85]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Initialize the OneHotEncoder
F = OneHotEncoder()

# Fit and transform the 'Geography' column (reshaping it to 2D)
geography_encoded = F.fit_transform(data[["Geography"]])

geography_encoded 
geography_encoded.toarray()
F.get_feature_names_out(["Geography"]) 

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [86]:
df=pd.DataFrame(geography_encoded.toarray(),columns=F.get_feature_names_out(["Geography"]))

df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [87]:
#combine with original data
new_df = pd.concat([data.drop('Geography',axis=1),df],axis=1)
new_df.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [88]:
X=new_df.drop('Exited',axis=1)
y=new_df['Exited']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [131]:
from tensorflow.keras.models import load_model
import pickle


# Load the OneHotEncoder for 'Geography'
with open('geography_encoded.pkl', 'rb') as file:
    g_geography_encoded = pickle.load(file)

# Load the LabelEncoder for 'Gender'
with open("label_encoder_gender.pkl", "rb") as file:
    label_encoder_gen = pickle.load(file)

# Load the scaler (StandardScaler, MinMaxScaler, etc.)
with open("scaler.pkl", 'rb') as file:
    scaler = pickle.load(file)

# Check if all models and encoders have loaded correctly

print(type(g_geography_encoded))
print(type(label_encoder_gen))
print(type(scaler))


<class 'scikeras.wrappers.KerasClassifier'>
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
<class 'sklearn.preprocessing._label.LabelEncoder'>
<class 'sklearn.preprocessing._data.StandardScaler'>


In [132]:
##define a function to create a ,model and try different parameters(keras clasfier)

def create_model(neuron=32,layers=1):
    model=Sequential()
    model.add(Dense(neuron,activation='relu',input_shape=(X_train.shape[1],)))#first layer

    for _ in range (layers-1):
        model.add(Dense(neuron,activation='relu'))#add layers

    
    model.add(Dense(1,activation='sigmoid'))#output layer

    model.compile(optimizer="adam",loss="binary_cross_entropy",metrics=["accuracy"])

    return model








In [133]:
##CREATE A CLASSIFIER=KERAS
model=KerasClassifier(layers=1,neuron=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)



In [134]:
param_grid = {
    'neurons': [16, 32],
    'layers': [1, 2],  # Ensure 'layers' is a valid parameter
    'batch_size': [10, 20],
    'epochs': [50, 100]
}

In [138]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best Score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

2024-09-22 16:31:11.660759: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-22 16:31:11.662989: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-22 16:31:11.669284: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-22 16:31:11.682539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-22 16:31:11.687815: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different 

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [120]:
# Assuming 'model' is your KerasClassifier instance
print(model.get_params().keys())

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'layers', 'neurin', 'class_weight'])
